In [16]:
from pathlib import Path
import csv
def read_sample_label(list_path):
        with open(list_path) as metalist:
            datalist = csv.reader(metalist, delimiter=',')
            return [(row[0], int(row[1]), row[2]) for row in datalist if datalist.line_num > 1]
def find_label(label_list, target_fname, target_slice):
        return next((label for fname, slice_val, label, fname_list in label_list if fname == target_fname and fname_list[target_slice] == str(target_slice)), None)


root = 'Z:\data/fastMRI\knee/retain_singlecoil_test/'
files = list(Path(root).iterdir())
for fname in sorted(files):
    print(fname)
    print(fname.name)




FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Z:\\data\\fastMRI\\knee\\retain_singlecoil_test'

In [ ]:
import os
import logging
import random
import torch
import pickle
import csv
import h5py
import yaml
import numpy as np
import xml.etree.ElementTree as etree
from pathlib import Path
from warnings import warn
from typing import NamedTuple, Dict, Any, Union, Optional, Callable, Tuple, Sequence
import pandas as pd

root = '../Dataset/label.csv'
data_root = 'Z:/data/fastMRI\knee\singlecoil_train'
def read_sample_label(list_path):
    # Assuming the CSV file has columns: 'file', 'slice', 'label'
    label_df = pd.read_csv(list_path, header=0, names=['file', 'slice', 'label'])
    return label_df

def find_label(label_list, target_fname, target_slice):
    # Assuming label_list is a DataFrame with columns: 'file', 'slice', 'label'
    filtered_rows = label_list.loc[(label_list['file'] == target_fname) & (label_list['slice'] == target_slice)]
    if not filtered_rows.empty:
        return filtered_rows['label'].values[0]
    else:
        return 0

def remove_h5_extension(fname):
    return os.path.splitext(fname.name)[0]

def et_query(
    root: etree.Element,
    qlist: Sequence[str],
    namespace: str = "http://www.ismrm.org/ISMRMRD",
) -> str:
    """
    ElementTree query function.

    This can be used to query an xml document via ElementTree. It uses qlist
    for nested queries.

    Args:
        root: Root of the xml to search through.
        qlist: A list of strings for nested searches, e.g. ["Encoding",
            "matrixSize"]
        namespace: Optional; xml namespace to prepend query.

    Returns:
        The retrieved data as a string.
    """
    s = "."
    prefix = "ismrmrd_namespace"

    ns = {prefix: namespace}

    for el in qlist:
        s = s + f"//{prefix}:{el}"

    value = root.find(s, ns)
    if value is None:
        raise RuntimeError("Element not found")

    return str(value.text)


def fetch_dir(key: str, data_config_file="fastmri_dirs.yaml") -> Path:
    """_summary_

    Args:
        key (_type_): key to retrieve path from data_config_file. Expected to be in
            ("knee_path", "brain_path", "log_path").
        data_config_file (str, optional): Default path config file to fetch path
            from. Defaults to "fastmri_dirs.yaml".

    Returns:
        Path: _description_
    """

    data_config_file = Path(data_config_file)
    if not data_config_file.is_file():
        default_config = {
            "knee_path": "Dataset/fastMRI/",
            "brain_path": "Dataset/FastMRI/",
            "log_path": "logs/",
        }

        with open(data_config_file, "w") as f:
            yaml.dump(default_config, f)

        data_dir = default_config[key]

        warn(
            f"Path config at {data_config_file.resolve()} does not exist. "
            "A template has been created for you. "
            "Please enter the directory paths for your system to have defaults."
        )
    else:
        with open(data_config_file, "r") as f:
            data_dir = yaml.safe_load(f)[key]

    return Path(data_dir)


class FastMRIRawDataSample(NamedTuple):
    """Basic data type for fastMRI raw data.

    Elements:
        fname: Path for each h5 file, Path
        slice_ind: slice index, int
        metadata: metadata for each volume, Dict
    """
    fname: Path
    slice_ind: int
    label: list
    metadata: Dict[str, Any]

def _retrieve_metadata(fname):
    """_summary_

    Args:
        fname (_type_):

    Returns:
        _type_: _description_
    """
    with h5py.File(fname, 'r') as hf:
        et_root = etree.fromstring(hf['ismrmrd_header'][()])

        enc = ["encoding", "encodedSpace", "matrixSize"]
        enc_size = (
            int(et_query(et_root, enc + ["x"])), #640
            int(et_query(et_root, enc + ["y"])), # 372
            int(et_query(et_root, enc + ["z"])), # 1
        )
        rec = ["encoding", "reconSpace", "matrixSize"]
        recon_size = (
            int(et_query(et_root, rec + ["x"])), # 320
            int(et_query(et_root, rec + ["y"])), # 320
            int(et_query(et_root, rec + ["z"])), # 1
        )

        lims = ["encoding", "encodingLimits", "kspace_encoding_step_1"]
        enc_limits_center = int(et_query(et_root, lims + ["center"])) # 167
        enc_limits_max = int(et_query(et_root, lims + ["maximum"])) + 1 # 334

        padding_left = enc_size[1] // 2 - enc_limits_center # 372 // 2 - 167 = 19
        padding_right = padding_left + enc_limits_max # 19 + 334 = 353

        num_slices = hf["kspace"].shape[0]

        metadata = {
            "padding_left": padding_left,
            "padding_right": padding_right,
            "encoding_size": enc_size,
            "recon_size": recon_size,
            **hf.attrs
        }

    return metadata, num_slices



raw_samples = []
# * if there is a filter
raw_sample_filter = None
if raw_sample_filter is None:
    raw_sample_filter = lambda raw_sample: True
else:
    raw_sample_filter = raw_sample_filter

label_list = read_sample_label(root)
files = list(Path(data_root).iterdir())
for fname in sorted(files):
    metadata, num_slices = _retrieve_metadata(fname)
    new_raw_samples = []
    for slice_ind in range(num_slices):
        label = find_label(label_list, remove_h5_extension(fname), slice_ind)
        print(label)
        raw_sample = FastMRIRawDataSample(fname, slice_ind, label, metadata)
        if raw_sample_filter(raw_sample):
            new_raw_samples.append(raw_sample)
    raw_samples += new_raw_samples
    # print(raw_samples)


In [ ]:
size_all =[]
for i in range(len(raw_samples)):
    fname, dataslice, label, metadata = raw_samples[i]
    if label == 1:
        # print(fname, dataslice, label, metadata)
        with h5py.File(fname, "r") as hf:
            kspace = hf["kspace"][dataslice]
            size_all.append(np.shape(hf["kspace"]))

In [ ]:
print(np.shape(size_all))
from matplotlib import pyplot as plt
new = np.array(size_all)

print(np.shape(new))

print(max(new[:,0]),max(new[:,1]),max(new[:,2]))
print(min(new[:,0]),min(new[:,1]),min(new[:,2]))
# Plotting a basic histogram
plt.hist(new[:,2], bins=100, color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Basic Histogram')

# Display the plot
plt.show()

In [ ]:
data = np.stack((kspace.real, kspace.imag), axis=-1)
print(data)
data = torch.from_numpy(data)
print(np.shape(data))
print(data)

In [ ]:
def apply_mask(data: torch.Tensor, mask_func: MaskFunc, offset: Optional[int]=None,
                seed: Optional[Union[int, Tuple[int, ...]]]=None, padding: Optional[Sequence[int]]=None):

    shape = (1,) * len(data.shape[:-3]) + tuple(data.shape[-3:]) # (1, 1, 640, 372, 2)
    print(shape)
    mask, num_low_freqs = mask_func(shape, offset, seed)
    if padding is not None:
        mask[..., :padding[0], :] = 0
        mask[..., padding[1]:, :] = 0

    # * add 0.0 removes the sign of the zeros
    masked_data = data * mask + 0.0

    return masked_data, mask, num_low_freqs

In [ ]:
from utils.fft import fft2c, ifft2c
from masking import create_mask_for_mask_type
mask = create_mask_for_mask_type('random', [0.08], [4])
mask_func = None
image = ifft2c(data)
noisy_masked_kspace, mask_, _ = apply_mask(data,mask, seed=42)

In [ ]:
print(np.shape(mask_))
print(np.shape(noisy_masked_kspace))

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(noisy_masked_kspace[:,:,0])

In [ ]:
for i in range[2:5]:
 print(i)

In [ ]:
import torch.nn as nn
loss = nn.BCELoss()
input = torch.randn(3, 2, requires_grad=True)
target = torch.rand(3, 2, requires_grad=False)
output = loss(torch.sigmoid(input), target)
output.backward()
print(input, target)

loss = nn.BCEWithLogitsLoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(input, target)
output.backward()
print(torch.sigmoid(input), target)

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
import torch
loss = nn.BCELoss()
original_labels = torch.tensor([0, 1, 0, 1, 0, 1])
one_hot_label = F.one_hot(original_labels , num_classes=2).float()
input = torch.randn(6, 2, requires_grad=True)
# loss = loss(torch.sigmoid(input), one_hot_label)
acc = (input.argmax(dim=-1) == original_labels).float().mean()
print(acc)

In [ ]:
import numpy as np

# Assuming your original labels are stored in a list or numpy array
original_labels = [0, 1, 0, 1, 0, 1]

# Convert binary labels to one-hot encoded labels
one_hot_labels = np.eye(2)[original_labels]

print("Original Labels:", original_labels)
print("One-Hot Encoded Labels:")
for label in one_hot_labels:
    print(label)


In [ ]:
import torch
import torchvision.models as models

# Load pre-trained ResNet-50
resnet50 = models.resnet50(pretrained=True)

# Access 'layer4'
layer4 = resnet50.layer4

# Now 'layer4' contains the fourth block of ResNet-50
print(layer4)


In [ ]:

import torch.nn as nn# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
print(input,target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

In [ ]:
import numpy as np

# Assuming your one-hot encoded labels are stored in a 2D array
one_hot_labels = np.array([[0.5, 0], [0, 1], [1, 0], [0, 1], [1, 0]])

# Convert one-hot encoded labels to original labels
original_labels = np.argmax(one_hot_labels, axis=1)

print("One-Hot Encoded Labels:")
for label in one_hot_labels:
    print(label)

print("Original Labels:", original_labels)


In [ ]:
import torch.nn.functional as F
input = torch.randn(3, 5, requires_grad=True)

target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
print(input, target)
loss.backward()

In [ ]:
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
loss = F.binary_cross_entropy_with_logits(input, target)
loss.backward()
print(input,target)

In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from sklearn.utils import shuffle
from torch.utils.data import DataLoader

len_ds = 60461
counts = {'A': 6775, 'B': 3609, 'C': 906}


def create_data(which_class: str):
    arr = np.zeros((1, len_ds))
    arr[:, :counts[which_class]] = 1
    return arr


data = {k: create_data(k) for k in counts}
data = np.concatenate((data['A'], data['B'], data['C']), axis=0)

label_df = pd.DataFrame(data).transpose()
label_df.columns = [*counts.keys()]
label_df = shuffle(label_df)
label_df.reset_index(inplace=True, drop=True)
print(label_df[label_df == 1].count())


class toy_dataset(Dataset):
    def __getitem__(self, index):
        return torch.tensor(label_df.iloc[index].values)

    def __len__(self):
        return len(label_df)


dataset = toy_dataset()


def calculateWeights(label_dict, d_set):
    arr = []
    for label, count in label_dict.items():
        weight = len(d_set) / count
        arr.append(weight)
    return arr


weights = calculateWeights(counts, dataset)
weights = torch.DoubleTensor(weights)
print('weights: ', weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(dataset), replacement=True)
trainloader = DataLoader(dataset, batch_size=50, sampler=sampler,
                         shuffle=False,
                         num_workers=0, pin_memory=False)

all_labels = torch.Tensor()
for labels in trainloader:
    all_labels = torch.cat((all_labels, labels.cpu()), 0)

print(all_labels.shape)


In [ ]:
print(all_labels[12])